1. 성별과 연령별 합격시간의 표를 참고하여 기존 marathon_2015_2017 데이터에 Qualifying 컬럼을 추가합니다 
1. 최종 기록이 해당 기록보다 늦으면 0, 빠르면 1 로 저장합니다
1. 처리가 완료된 파일을 marathon_2015_2017_qualifying.csv 파일로 저장합니다
1. 저장된 파일을 불러와서 학습하고 특정 참가자의 Qualifying 을 예측합니다

In [7]:
import pandas as pd
import numpy as np
marathon_qualifying_time = pd.read_csv("./marathon_qualifying_time.csv")
# marathon_qualifying_time.head()

In [8]:
# 나이를 제외한 남녀 기록들만 추출
qualifying_time = pd.DataFrame(marathon_qualifying_time , columns=['F',  'M'])
#qualifying_time.head()

In [9]:
# 기록들을 모두 초(s)로 변환
qualifying_time['F'] = pd.to_timedelta(qualifying_time['F'])
qualifying_time['M'] = pd.to_timedelta(qualifying_time['M'])
qualifying_time['F'] = qualifying_time['F'].astype('m8[s]').astype(np.int64)
qualifying_time['M'] = qualifying_time['M'].astype('m8[s]').astype(np.int64)

In [11]:
# qualifying_time.head()

In [12]:
# 마라톤 기록 불러와서, 성별 1, 0 으로 변경  qualifying 컬럼 추가
marathon_2015_2017 = pd.read_csv("./marathon_2015_2017.csv")
marathon_2015_2017['M/F'] = marathon_2015_2017['M/F'].map({'M': 1, 'F': 0})
# 성별, 나이 페이스 최종기록 년도 qualifying  컬럼을 갖는 빈 데이터 프레임 생성
marathon_2015_2017_qualifying = pd.DataFrame( \
        columns=['M/F',  'Age',  'Pace',  'Official Time', 'Year', 'qualifying' ])

In [13]:
# qualifying_time 데이터를 리스트로 변환
qualifying_time_list = qualifying_time.values.tolist()

In [14]:
for index, record in marathon_2015_2017.iterrows():
    # qualifying_time_list[ record.Age-18 ][ record['M/F'] ] 
    # -> 현재 레코드에서 나이값과 성별값으로 행열 검색의 결과
    qualifying_standard = qualifying_time_list[record.Age-18][record['M/F']]
    # 현재 레코드의 최종 기록이 qualifying_standard 빠르면 qualifying_status = 1, 
    # 아니면 0 저장
    qualifying_status = 1
    if (record['Official Time'] > qualifying_standard): 
        qualifying_status = 0
    
    # 레코드 각 필요 값들과 방금 계산한 qualifying_status 값을 이용해서 상단에 
    # 생성해 놓은 marathon_2015_2017_qualifying 에 추가( 딕셔너리 -> 데이터프레임)
    marathon_2015_2017_qualifying = marathon_2015_2017_qualifying.append(
        {'M/F':record['M/F'], 'Age':record['Age'], 'Pace':record['Pace'],
         'Official Time':record['Official Time'], 'Year':record['Year'],
         'qualifying':qualifying_status }, ignore_index=True )
# 최종 완성된 데이터 프레임을 파일로 저장합니다
marathon_2015_2017_qualifying.to_csv("./marathon_2015_2017_qualifying.csv", \
                                                     index = None, header=True)